In [ ]:
import pandas as pd
import numpy as np
import talib as ta
import os

# --------------------------
# 配置参数（核心放宽优化）
# --------------------------
CONFIG = {
    "factortable_path": r'./factortable.parquet',
    "selection_result_path": r'./short_term_selection_optimized.csv',
    "daily_result_dir": r'./daily_short_term_optimized',
    "log_path": r'./short_term_selection_optimized_log.txt',
    "top_n": 50,  # 目标每日20-50只，取前50只（避免过多分散）
    # 优化：放宽形态筛选（核心调整区）
    "short_term_shape": {
        "consecutive_up_days_min": 2,        # 放宽：连续上涨≥2天（原3天）
        "consecutive_up_days_max": 8,         # 放宽：连续上涨≤8天（原7天）
        "rise_ratio_30d_min": 0.05,          # 放宽：30日涨幅≥5%（原8%）
        "rise_ratio_30d_max": 0.5,           # 放宽：30日涨幅≤50%（原40%）
        "daily_rise_min": -0.01,             # 放宽：单日涨幅≥-1%（原-0.5%，允许小幅回调）
        "daily_rise_max": 0.05,              # 放宽：单日涨幅≤5%（原3.5%，保留上限防暴涨）
        "volume_ratio_min": 0.7,             # 放宽：量能比≥0.7（原0.9，允许小幅回落）
        "volume_ratio_consecutive_min": 1,   # 大幅放宽：连续≥1天量能比≥1.0（原2天）
        "rsi_safe_max": 70,                  # 放宽：RSI≤70（原65，允许轻度超买）
        "ma_trend": True,                    # 保留：均线多头（策略核心）
        # 放宽：竞价筛选（核心调整）
        "high_open_min": 0.003,               # 放宽：高开≥0.3%（原0.5%）
        "high_open_max": 0.03,                # 放宽：高开≤3%（原2%，扩大区间）
        "auction_volume_ratio_min": 0.02     # 放宽：竞价量比≥2%（原3%）
    },
    # 排序权重：突出量价配合和竞价（不变）
    "sort_weights": {
        "auction_score": 0.35,        # 竞价得分（权重最高，聚焦资金热度）
        "price_volume_score": 0.3,    # 量价配合（核心，验证趋势真实性）
        "trend_strength_score": 0.2,  # 趋势强度（辅助，保证上涨惯性）
        "risk_filter_score": 0.15     # 风险过滤（辅助，控制回撤）
    }
}

# --------------------------
# 工具函数（不变）
# --------------------------
def init_environment():
    os.makedirs(CONFIG["daily_result_dir"], exist_ok=True)
    with open(CONFIG["log_path"], 'w', encoding='utf-8') as f:
        f.write(f"【短线形态放宽版选股启动】{pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    log_msg(f"✅ 环境初始化完成，每日结果目录：{CONFIG['daily_result_dir']}")

def log_msg(msg):
    timestamp = pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')
    log_line = f"[{timestamp}] {msg}"
    print(log_line)
    with open(CONFIG["log_path"], 'a', encoding='utf-8') as f:
        f.write(log_line + "\n")

def validate_data(df):
    required_cols = [
        'stock_code', 'date', 'close', 'open', 'volume', 'high', 'low',
        'consec 'high', 'low',
        'consecutive_up_days', 'is_high_open', 'ma5', 'ma20',
        'macd_line', 'signal_line', 'macd_hist', 'volume_ratio_5d',
        'high_30d', 'pullback_ratio', 'pullback_days', 'bollinger_lower', 'rsi14',
        'auction_rise_ratio', 'auction_volume_ratio'
    ]
    missing_cols = [col for col in required_cols if col not in df.columns]
    if missing_cols:
        raise ValueError(f"Factortable缺少字段：{missing_cols}")
    log_msg(f"✅ 字段验证通过，共{len(df)}条记录")
    return df

# --------------------------
# 步骤1：计算衍生指标（优化连续量能逻辑）
# --------------------------
def pre_calculate_indicators(df):
    log_msg("开始计算放宽版衍生指标...")
    
    # 1. 30日涨幅（放宽阈值后同步调整计算逻辑）
    def calc_30d_rise(group):
        group['rise_ratio_30d'] = (group['close'] - group['close'].shift(30)) / \
                                 group['close'].shift(30).replace(0, 0.0001)
        return group['rise_ratio_30d'].clip(0, 1.0).fillna(0)  # 保留上限剪辑，避免极端值
    df['rise_ratio_30d'] = df.groupby('stock_code', group_keys=False).apply(calc_30d_rise)
    
    # 2. 每日涨幅（放宽区间后同步计算）
    df['daily_rise_ratio'] = (df['close'] - df['open']) / df['open'].replace(0, 0.0001)
    
    # 3. 连续量能达标记（核心优化：允许1天中断，更贴合实际行情）
    def calc_consecutive_volume(group):
        # 先标记量能是否达标（≥1.0）
        group['vol_over'] = (group['volume_ratio_5d'] >= 1.0).astype(int)
        # 基础连续计数（原逻辑）
        vol_diff = group['vol_over'].ne(group['vol_over'].shift()).cumsum()
        group['consecutive_volume_days'] = group['vol_over'].groupby(vol_diff).cumsum()
        
        # 优化：允许1天中断（当天不达标但前一天达标，保留前一天计数）
        for i in range(1, len(group)):
            prev_count = group.iloc[i-1]['consecutive_volume_days']
            curr_over = group.iloc[i]['vol_over']
            if curr_over == 0 and prev_count >= 1:
                group.iloc[i, group.columns.get_loc('consecutive_volume_days')] = prev_count
        return group.drop(columns=['vol_over'])  # 删除临时列
    df = df.groupby('stock_code', group_keys=False).apply(calc_consecutive_volume)
    
    # 4. 价格在5日线上方标记（保留，确保趋势稳健）
    df['price_above_ma5'] = (df['close'] >= df['ma5']).astype(int)
    
    log_msg("✅ 放宽版衍生指标计算完成")
    return df

# --------------------------
# 步骤2：形态筛选（应用放宽后参数）
# --------------------------
def filter_single_shape(daily_df):
    params = CONFIG["short_term_shape"]
    daily_df = daily_df.copy()
    
    # 核心：应用放宽后的筛选条件
    # 条件1：连续上涨2-8天（放宽区间）
    cond1 = daily_df['consecutive_up_days'].between(
        params['consecutive_up_days_min'], 
        params['consecutive_up_days_max']
    )
    # 条件2：30日涨幅5%-50%（放宽区间）
    cond2 = daily_df['rise_ratio_30d'].between(
        params['rise_ratio_30d_min'], 
        params['rise_ratio_30d_max']
    )
    # 条件3：单日涨幅-1%-5%（允许小幅回调，保留上限）
    cond3 = daily_df['daily_rise_ratio'].between(
        params['daily_rise_min'], 
        params['daily_rise_max']
    )
    # 条件4：连续≥1天量能比≥1.0（大幅放宽）
    cond4 = daily_df['consecutive_volume_days'] >= params['volume_ratio'] >= params['volume_ratio_consecutive_min']
    # 条件5：价格在5日线上方（保留核心逻辑）
    cond5 = daily_df['price_above_ma5'] == 1
    # 条件6：量能比≥0.7（放宽量能门槛）
    cond6 = daily_df['volume_ratio_5d'] >= params['volume_ratio_min']
    # 条件7：RSI≤70（允许轻度超买）
    cond7 = daily_df['rsi14'] <= params['rsi_safe_max']
    # 条件8：均线多头（保留核心逻辑）
    cond8 = (daily_df['ma5'] >= daily_df['ma20']) if params['ma_trend'] else True
    
    # 条件9-11：放宽竞价筛选
    cond9 = daily_df['auction_rise_ratio'].between(
        params['high_open_min'], 
        params['high_open_max']
    )  # 高开0.3%-3%
    cond10 = daily_df['auction_volume_ratio'] >= params['auction_volume_ratio_min']  # 竞价量比≥2%
    cond11 = daily_df['is_high_open'] == True
    
    # 组合筛选（11个条件，门槛降低但核心逻辑保留）
    total_cond = cond1 & cond2 & cond3 & cond4 & cond5 & cond6 & cond7 & cond8 & cond9 & cond10 & cond11
    filtered_df = daily_df[total_cond].copy()
    
    # 动态提示：根据选股数量给出调整建议
    log_msg(f"放宽版形态筛选：符合条件{len(filtered_df)}只（目标20-50）")
    if len(filtered_df) < 20:
        log_msg(f"⚠️ 标的仍不足，建议进一步放宽：30日涨幅≥3% 或 竞价量比≥1% 或 连续量能≥1天")
    elif len(filtered_df) > 50:
        log_msg(f"⚠️ 标的过多，建议小幅收紧：单日涨幅≤4% 或 连续量能≥2天 或 竞价量比≥3%")
    return filtered_df

# --------------------------
# 步骤3：排序优化（权重不变，适配放宽后标的）
# --------------------------
def score_by_short_term_factors(filtered_df):
    if len(filtered_df) == 0:
        return filtered_df
    df = filtered_df.copy()
    weights = CONFIG["sort_weights"]
    
    # 1. 竞价得分（适配放宽后的高开区间：0.3%-3%）
    df['auction_rise_score'] = ((df['auction_rise_ratio'] - 0.003) / (0.03 - 0.003 + 1e-8)) * 50
    df['auction_vol_score'] = ((df['auction_volume_ratio'] - 0.02) / (0.1 - 0.02 + 1e-8)) * 50  # 适配2%-10%区间
    df['auction_score'] = (df['auction_rise_score'] + df['auction_vol_score']).clip(0, 100)
    
    # 2. 量价配合得分（适配放宽后的量能和涨幅区间）
    df['volume_strength_score'] = ((df['volume_ratio_5d'] - 0.7) / (2.0 - 0.7 + 1e-8)) * 50  # 0.7-2.0映射
    df['price_strength_score'] = ((df['daily_rise_ratio'] - (-0.01)) / (0.05 - (-0.01) + 1e-8)) * 50  # -1%-5%映射
    df['price_volume_score'] = (df['volume_strength_score'] + df['price_strength_score']).clip(0, 100)
    
    # 3. 趋势强度得分（适配放宽后的连续上涨天数：2-8天）
    up_days_max = df['consecutive_up_days'].max() if df['consecutive_up_days'].max() > 0 else 1
    df['up_days_score'] = (df['consecutive_up_days'] / up_days_max) * 60
    df['ma_strength_score'] = ((df['ma5'] - df['ma20']) / df['ma20'].replace(0, 0.0001) * 1000).clip(0, 40)
    df['trend_strength_score'] = df['up_days_score'] + df['ma_strength_score']
    
    # 4. 风险过滤得分（适配放宽后的RSI上限：70）
    df['rsi_risk_score'] = 100 - (abs(df['rsi14'] - 50) / 40) * 100  # 50±40区间（10-90）映射
    df['bollinger_risk_score'] = (1.2 - df['close']/df['bollinger_lower']) / (0.2 + 1e-8) * 100
    df['risk_filter_score'] = (df['rsi_risk_score'] + df['bollinger_risk_score']).clip(0, 100) / 2
    
    # 综合得分（权重不变，突出核心维度）
    df['total_score'] = (
        df['auction_score'] * weights['auction_score'] +
        df['price_volume_score'] * weights['price_volume_score'] +
        df['trend_strength_score'] * weights['trend_strength_score'] +
        df['risk_filter_score'] * weights['risk_filter_score']
    )
    
    # 取前N只（适配放宽后的top_n=50）
    df = df.sort_values(by='total_score', ascending=False).head(CONFIG["top_n"]).reset_index(drop=True)
    log_msg(f"放宽版排序完成：最高得分{df['total_score'].max():.2f}，前20平均得分{df['total_score'].head(20).mean():.2f}")
    return df

# --------------------------
# 步骤4：单交易日处理（放宽数据异常阈值）
# --------------------------
def process_single_trade_date(df, trade_date):
    log_msg(f"\n===== 处理交易日：{trade_date.strftime('%Y-%m-%d')} =====")
    daily_df = df[df['date'].dt.date == trade_date].copy()
    # 放宽异常阈值：当日数据<500条才判定为异常（原1000条）
    if len(daily_df) < 500:
        log_msg(f"⚠️ 当日数据异常（记录数{len(daily_df)}<500），跳过")
        return pd.DataFrame()
    
    filtered_df = filter_single_shape(daily_df)
    if len(filtered_df) == 0:
        log_msg("⚠️ 无符合标的，跳过")
        return pd.DataFrame()
    
    ranked_df = score_by_short_term_factors(filtered_df)
    
    # 保留关键字段（便于后续回测和IC/IR分析）
    result_cols = [
        'stock_code', 'date', 'close', 'open', 'volume',
        'consecutive_up_days', 'rise_ratio_30d', 'daily_rise_ratio',
        'volume_ratio_5d', 'consecutive_volume_days',
        'auction_rise_ratio', 'auction_volume_ratio', 'rsi14',
        'ma5', 'ma20', 'total_score', 'auction_score', 'price_volume_score'
    ]
    top_df = ranked_df[result_cols].reset_index(drop=True)
    top_df['trade_date'] = trade_date.strftime('%Y-%m-%d')
    
    # 保存每日结果
    date_str = trade_date.strftime('%Y%m%d')
    daily_save_path = os.path.join(CONFIG["daily_result_dir"], f"short_term_optimized_{date_str}.csv")
    top_df.to_csv(daily_save_path, index=False, encoding='utf-8-sig')
    log_msg(f"✅ 当日选股完成：{len(top_df)}只标的（目标20-50）")
    return top_df

# --------------------------
# 主流程（不变，适配放宽版参数）
# --------------------------
def run_short_term_selection():
    try:
        init_environment()
        # 加载因子表并预处理
        log_msg("加载factortable数据...")
        df = pd.read_parquet(CONFIG["factortable_path"])
        df['date'] = pd.to_datetime(df['date'])
        # 数据验证+衍生指标计算
        df = validate_data(df)
        df = pre_calculate_indicators(df)
        
        # 遍历所有交易日选股
        trade_dates = sorted(df['date'].dt.date.unique())
        log_msg(f"检测到{len(trade_dates)}个交易日，开始放宽版选股...")
        
        all_results = []
        for trade_date in trade_dates:
            daily_result = process_single_trade_date(df, trade_date)
            if not daily_result.empty:
                all_results.append(daily_result)
        
        # 保存最终结果
        if all_results:
            final_result = pd.concat(all_results, ignore_index=True)
            final_result.to_csv(CONFIG["selection_result_path"], index=False, encoding='utf-8-sig')
            log_msg(f"\n✅ 放宽版选股完成！累计{len(final_result)}条记录，路径：{CONFIG['selection_result_path']}")
            log_msg(f"📊 平均每日选股：{len(final_result)/len(trade_dates):.1f}只（目标20-50）")
        else:
            log_msg(f"\n⚠️ 无选股结果，请按日志提示进一步放宽参数")
    except Exception as e:
        log_msg(f"❌ 选股失败：{str(e)}")
        raise

if __name__ == "__main__":
    run_short_term_selection()

SyntaxError: '[' was never closed (3817809741.py, line 58)